In [2]:
!pip install pytorch-tabnet -qq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.7 MB/s eta 0:00:00


In [3]:
!pip install huggingface_hub -qq


In [4]:
!pip install torch -qq


In [5]:
!pip install readability -qq

  Preparing metadata (setup.py) ... done


In [6]:


import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd

# Download the necessary VADER lexicon
nltk.download('vader_lexicon')

# Initialize VADER SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()


import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from collections import Counter
import nltk
import string
import readability
import spacy
from tqdm import tqdm  # Import tqdm for progress bar

# Download necessary NLTK resources and spaCy model
nltk.download('vader_lexicon')
nltk.download('stopwords')
spacy.cli.download("en_core_web_sm")  # Download the small English model for spaCy

# Initialize SentimentIntensityAnalyzer, stopwords, and spaCy model
sia = SentimentIntensityAnalyzer()
stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")

# Modify tqdm to work with pandas apply
tqdm.pandas()

def extract_features(df, text_column='text', title_column='title', subject_column='subject', date_column='date'):

    # Initialize feature DataFrame
    features = pd.DataFrame()

    features['text_length'] = df[text_column].progress_apply(len)
    features['title_length'] = df[title_column].progress_apply(len)
    features['word_count'] = df[text_column].progress_apply(lambda x: len(x.split()))
    features['title_word_count'] = df[title_column].progress_apply(lambda x: len(x.split()))
    features['avg_word_length'] = df[text_column].progress_apply(lambda x: np.mean([len(word) for word in x.split()]))
    features['unique_word_ratio'] = df[text_column].progress_apply(lambda x: len(set(x.split())) / len(x.split()))

    ## Sentence-level metrics
    features['sentence_count'] = df[text_column].progress_apply(lambda x: x.count('.') + x.count('!') + x.count('?'))
    features['avg_sentence_length'] = features['word_count'] / (features['sentence_count'] + 1e-5)

    ## Punctuation metrics
    features['punctuation_count'] = df[text_column].progress_apply(lambda x: sum(1 for char in x if char in string.punctuation))
    features['title_punctuation_count'] = df[title_column].progress_apply(lambda x: sum(1 for char in x if char in string.punctuation))

    ## Sentiment analysis
    features['text_sentiment'] = df[text_column].progress_apply(lambda x: sia.polarity_scores(x)['compound'])
    features['title_sentiment'] = df[title_column].progress_apply(lambda x: sia.polarity_scores(x)['compound'])

    ## Stopword ratio
    features['stopword_ratio'] = df[text_column].progress_apply(
        lambda x: len([word for word in x.split() if word.lower() in stop_words]) / len(x.split())
    )

    ## Capitalization metrics
    features['uppercase_word_ratio'] = df[text_column].progress_apply(
        lambda x: len([word for word in x.split() if word.isupper()]) / len(x.split())
    )

    vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=500)
    ngram_matrix = vectorizer.fit_transform(df[text_column])
    features['ngram_mean'] = ngram_matrix.mean(axis=1).A1
    features['ngram_std'] = np.std(ngram_matrix.toarray(), axis=1)  # Convert sparse matrix to dense

    ## Readability scores
    def calculate_readability(text):
        try:
            r = readability.getmeasures(text, lang='en')
            return r['readability grades']['FleschReadingEase']
        except:
            return np.nan
    features['flesch_reading_ease'] = df[text_column].progress_apply(calculate_readability)

    ## Clickbait detection based on sentiment
    def calculate_word_sentiment(word):
        # Get the sentiment polarity of each word
        return sia.polarity_scores(word)['compound']

    def contains_clickbait(title, sentiment_threshold=0.5):
        words = title.split()
        for word in words:
            word_sentiment = calculate_word_sentiment(word)
            if abs(word_sentiment) >= sentiment_threshold:
                return 1  # Indicates clickbait
        return 0  # No clickbait

    features['contains_clickbait'] = df[title_column].progress_apply(contains_clickbait)

    if subject_column in df.columns:
        features['subject_encoded'] = pd.factorize(df[subject_column])[0]

    return features


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.10/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import pandas as pd
import numpy as np 
test="/kaggle/input/222222/test.tsv"
train="/kaggle/input/1111111/train.tsv"

In [8]:
test_df=pd.read_csv(test,sep="\t")
train_df=pd.read_csv(train,sep="\t")

In [9]:
train_label=train_df["label"]
test_label=test_df["label"]

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import accuracy_score
import torch


In [11]:
test_features=extract_features(test_df)
train_features=extract_features(train_df)

100%|██████████| 30000/30000 [00:05<00:00, 5355.45it/s]


In [12]:
test_features=test_features.values
train_features=train_features.values

In [13]:
train_label=train_label.values
test_label=test_label.values

In [14]:
print(len(train_label))
len(train_features)

30000


30000

In [19]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

# Define the classifier with enhanced parameters
clf = TabNetClassifier(
    n_d=32,  # Number of feature representation neurons
    n_a=32,  # Number of attention neurons
    n_steps=7,  # More decision steps for better reasoning
    gamma=1.5,  # Feature selection relaxation parameter
    lambda_sparse=1e-3,  # Sparsity regularization to avoid overfitting
    optimizer_fn=torch.optim.AdamW,  # Optimizer for better gradient updates
    optimizer_params={
        "lr": 1e-4,  # Lower learning rate for fine-tuning
        "weight_decay": 1e-6  # Regularization to reduce overfitting
    },
    scheduler_params={
        "step_size": 25,  # Frequency of learning rate decay
        "gamma": 0.85  # Decay factor
    },
    scheduler_fn=torch.optim.lr_scheduler.StepLR,  # Scheduler for dynamic learning rate adjustment
    mask_type="sparsemax",  # Stronger feature sparsity compared to softmax
    device_name="cuda" if torch.cuda.is_available() else "cpu"  # Auto-detect device for computation
)

In [29]:
from sklearn.model_selection import KFold

combined_features = np.concatenate([train_features, test_features], axis=0)
combined_labels = np.concatenate([train_label, test_label], axis=0)


In [31]:

# Perform K-fold cross-validation and store results
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_train_valid_data = []

for train_index, valid_index in kf.split(combined_features):
    X_train, X_valid = combined_features[train_index], combined_features[valid_index]
    y_train, y_valid = combined_labels[train_index], combined_labels[valid_index]
    
    fold_train_valid_data.append((X_train, X_valid, y_train, y_valid))

# Now train on the entire dataset and validate using all folds
all_train_data = np.concatenate([X_train for X_train, _, _, _ in fold_train_valid_data], axis=0)
all_train_labels = np.concatenate([y_train for _, _, y_train, _ in fold_train_valid_data], axis=0)


In [32]:


clf.fit(
    X_train=all_train_data, 
    y_train=all_train_labels,
    eval_set=[(combined_features, combined_labels)],  # Evaluation on all combined data
    eval_name=['valid'],
    eval_metric=['accuracy'],
    max_epochs=200, 
    patience=30,  
    batch_size=256, 
    virtual_batch_size=128,  
    num_workers=4, 
    drop_last=True 
)



epoch 0  | loss: 1.25891 | valid_accuracy: 0.58314 |  0:00:28s
epoch 1  | loss: 0.73319 | valid_accuracy: 0.71456 |  0:00:58s
epoch 2  | loss: 0.58952 | valid_accuracy: 0.77947 |  0:01:27s
epoch 3  | loss: 0.51143 | valid_accuracy: 0.81846 |  0:01:56s
epoch 4  | loss: 0.46246 | valid_accuracy: 0.83835 |  0:02:25s
epoch 5  | loss: 0.42773 | valid_accuracy: 0.85047 |  0:02:54s
epoch 6  | loss: 0.40518 | valid_accuracy: 0.85831 |  0:03:23s
epoch 7  | loss: 0.38481 | valid_accuracy: 0.86803 |  0:03:52s
epoch 8  | loss: 0.37137 | valid_accuracy: 0.8719  |  0:04:21s
epoch 9  | loss: 0.35295 | valid_accuracy: 0.87749 |  0:04:50s
epoch 10 | loss: 0.34325 | valid_accuracy: 0.87979 |  0:05:19s
epoch 11 | loss: 0.33085 | valid_accuracy: 0.88533 |  0:05:48s
epoch 12 | loss: 0.32244 | valid_accuracy: 0.88907 |  0:06:16s
epoch 13 | loss: 0.31488 | valid_accuracy: 0.89484 |  0:06:47s
epoch 14 | loss: 0.30305 | valid_accuracy: 0.90099 |  0:07:18s
epoch 15 | loss: 0.29367 | valid_accuracy: 0.90412 |  0

KeyboardInterrupt: 

In [33]:
clf.save_model('tabnet_model_Final')


Successfully saved model at tabnet_model_Final.zip


'tabnet_model_Final.zip'

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

def evaluate_tabnet_model(model, X_test, y_test, multi_class='ovr'):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)  
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    if len(set(y_test)) > 2:  
        aucroc = roc_auc_score(y_test, y_pred_proba, multi_class=multi_class, average='weighted')
    else: 
        aucroc = roc_auc_score(y_test, y_pred_proba[:, 1])
    
    print("Model Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC-ROC: {aucroc:.4f}")
    
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)


In [36]:

evaluate_tabnet_model(clf, test_features,test_label)

Model Evaluation Metrics:
Accuracy: 0.9262
Precision: 0.9285
Recall: 0.9262
F1 Score: 0.9262
AUC-ROC: 0.9793

Confusion Matrix:
[[3834  450]
 [ 160 3823]]


In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "Advik-7/Fake-News" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

nlp_model = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    padding=True,
    max_length=512,
    device=0
)


In [ ]:
def run_both_models(features,text_list):
    features_prob=clf.predict(features)
        # Initialize a list to store text probabilities
    text_prob = []
    
    for text in text_list:
        result = nlp_model(text)
        label = result[0]['label']
        if label == 'LABEL_0':
            text_prob.append(0)
        elif label == 'LABEL_1':
            text_prob.append(1)
    return features_prob,text_prob
    
    

In [133]:
features_prob1,text_prob1=run_both_models(test_features,train_text_TT)

In [140]:
import pandas as pd

# Assuming features_prob1 and text_prob1 are lists
df = pd.DataFrame({
    'label': train_label
})

# Save the dataframe to a CSV file
df.to_csv('lablll.csv', index=False)

print("Data saved successfully as 'features_and_text_probs.csv'")


Data saved successfully as 'features_and_text_probs.csv'


In [110]:
train_label_TT=train_df["label"]

In [46]:
train_text_TT=train_df["text"]

In [101]:

first_row = train_tabular_TT.iloc[0, :]


In [100]:
train_tabular_TT=extract_features(train_df)

100%|██████████| 30000/30000 [00:05<00:00, 5362.90it/s]
